
<br>
========================<br>
Partial Dependence Plots<br>
========================<br>
Partial dependence plots show the dependence between the target function [2]_<br>
and a set of 'target' features, marginalizing over the values of all other<br>
features (the complement features). Due to the limits of human perception, the<br>
size of the target feature set must be small (usually, one or two) thus the<br>
target features are usually chosen among the most important features.<br>
This example shows how to obtain partial dependence plots from a<br>
:class:`~sklearn.neural_network.MLPRegressor` and a<br>
:class:`~sklearn.ensemble.HistGradientBoostingRegressor` trained on the<br>
California housing dataset. The example is taken from [1]_.<br>
The plots show four 1-way and two 1-way partial dependence plots (omitted for<br>
:class:`~sklearn.neural_network.MLPRegressor` due to computation time). The<br>
target variables for the one-way PDP are: median income (`MedInc`), average<br>
occupants per household (`AvgOccup`), median house age (`HouseAge`), and<br>
average rooms per household (`AveRooms`).<br>
.. [1] T. Hastie, R. Tibshirani and J. Friedman, "Elements of Statistical<br>
       Learning Ed. 2", Springer, 2009.<br>
.. [2] For classification you can think of it as the regression score before<br>
       the link function.<br>


In [ ]:
print(__doc__)

In [ ]:
from time import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import make_pipeline

In [ ]:
from sklearn.inspection import partial_dependence
from sklearn.inspection import plot_partial_dependence
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import fetch_california_housing

############################################################################<br>
California Housing data preprocessing<br>
-------------------------------------<br>
<br>
Center target to avoid gradient boosting init bias: gradient boosting<br>
with the 'recursion' method does not account for the initial estimator<br>
(here the average target, by default)

In [ ]:
cal_housing = fetch_california_housing()
X = pd.DataFrame(cal_housing.data, columns=cal_housing.feature_names)
y = cal_housing.target

In [ ]:
y -= y.mean()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,
                                                    random_state=0)

############################################################################<br>
Partial Dependence computation for multi-layer perceptron<br>
---------------------------------------------------------<br>
<br>
Let's fit a MLPRegressor and compute single-variable partial dependence<br>
plots

In [ ]:
print("Training MLPRegressor...")
tic = time()
est = make_pipeline(QuantileTransformer(),
                    MLPRegressor(hidden_layer_sizes=(50, 50),
                                 learning_rate_init=0.01,
                                 early_stopping=True))
est.fit(X_train, y_train)
print("done in {:.3f}s".format(time() - tic))
print("Test R2 score: {:.2f}".format(est.score(X_test, y_test)))

############################################################################<br>
We configured a pipeline to scale the numerical input features and tuned the<br>
neural network size and learning rate to get a reasonable compromise between<br>
training time and predictive performance on a test set.<br>
<br>
Importantly, this tabular dataset has very different dynamic ranges for its<br>
features. Neural networks tend to be very sensitive to features with varying<br>
scales and forgetting to preprocess the numeric feature would lead to a very<br>
poor model.<br>
<br>
It would be possible to get even higher predictive performance with a larger<br>
neural network but the training would also be significantly more expensive.<br>
<br>
Note that it is important to check that the model is accurate enough on a<br>
test set before plotting the partial dependence since there would be little<br>
use in explaining the impact of a given feature on the prediction function of<br>
a poor model.<br>
<br>
Let's now compute the partial dependence plots for this neural network using<br>
the model-agnostic (brute-force) method:

In [ ]:
print('Computing partial dependence plots...')
tic = time()
# We don't compute the 2-way PDP (5, 1) here, because it is a lot slower
# with the brute method.
features = ['MedInc', 'AveOccup', 'HouseAge', 'AveRooms']
plot_partial_dependence(est, X_train, features,
                        n_jobs=3, grid_resolution=20)
print("done in {:.3f}s".format(time() - tic))
fig = plt.gcf()
fig.suptitle('Partial dependence of house value on non-location features\n'
             'for the California housing dataset, with MLPRegressor')
fig.subplots_adjust(hspace=0.3)

############################################################################<br>
Partial Dependence computation for Gradient Boosting<br>
----------------------------------------------------<br>
<br>
Let's now fit a GradientBoostingRegressor and compute the partial dependence<br>
plots either or one or two variables at a time.

In [ ]:
print("Training GradientBoostingRegressor...")
tic = time()
est = HistGradientBoostingRegressor()
est.fit(X_train, y_train)
print("done in {:.3f}s".format(time() - tic))
print("Test R2 score: {:.2f}".format(est.score(X_test, y_test)))

############################################################################<br>
Here, we used the default hyperparameters for the gradient boosting model<br>
without any preprocessing as tree-based models are naturally robust to<br>
monotonic transformations of numerical features.<br>
<br>
Note that on this tabular dataset, Gradient Boosting Machines are both<br>
significantly faster to train and more accurate than neural networks. It is<br>
also significantly cheaper to tune their hyperparameters (the default tend to<br>
work well while this is not often the case for neural networks).<br>
<br>
Finally, as we will see next, computing partial dependence plots tree-based<br>
models is also orders of magnitude faster making it cheap to compute partial<br>
dependence plots for pairs of interacting features:

In [ ]:
print('Computing partial dependence plots...')
tic = time()
features = ['MedInc', 'AveOccup', 'HouseAge', 'AveRooms',
            ('AveOccup', 'HouseAge')]
plot_partial_dependence(est, X_train, features,
                        n_jobs=3, grid_resolution=20)
print("done in {:.3f}s".format(time() - tic))
fig = plt.gcf()
fig.suptitle('Partial dependence of house value on non-location features\n'
             'for the California housing dataset, with Gradient Boosting')
fig.subplots_adjust(wspace=0.4, hspace=0.3)

############################################################################<br>
Analysis of the plots<br>
---------------------<br>
<br>
We can clearly see that the median house price shows a linear relationship<br>
with the median income (top left) and that the house price drops when the<br>
average occupants per household increases (top middle).<br>
The top right plot shows that the house age in a district does not have<br>
a strong influence on the (median) house price; so does the average rooms<br>
per household.<br>
The tick marks on the x-axis represent the deciles of the feature values<br>
in the training data.<br>
<br>
We also observe that :class:`~sklearn.neural_network.MLPRegressor` has much<br>
smoother predictions than<br>
:class:`~sklearn.ensemble.HistGradientBoostingRegressor`. For the plots to be<br>
comparable, it is necessary to subtract the average value of the target<br>
``y``: The 'recursion' method, used by default for<br>
:class:`~sklearn.ensemble.HistGradientBoostingRegressor`, does not account<br>
for the initial predictor (in our case the average target). Setting the<br>
target average to 0 avoids this bias.<br>
<br>
Partial dependence plots with two target features enable us to visualize<br>
interactions among them. The two-way partial dependence plot shows the<br>
dependence of median house price on joint values of house age and average<br>
occupants per household. We can clearly see an interaction between the<br>
two features: for an average occupancy greater than two, the house price is<br>
nearly independent of the house age, whereas for values less than two there<br>
is a strong dependence on age.

############################################################################<br>
3D interaction plots<br>
--------------------<br>
<br>
Let's make the same partial dependence plot for the 2 features interaction,<br>
this time in 3 dimensions.

In [ ]:
fig = plt.figure()

In [ ]:
features = ('AveOccup', 'HouseAge')
pdp, axes = partial_dependence(est, X_train, features=features,
                               grid_resolution=20)
XX, YY = np.meshgrid(axes[0], axes[1])
Z = pdp[0].T
ax = Axes3D(fig)
surf = ax.plot_surface(XX, YY, Z, rstride=1, cstride=1,
                       cmap=plt.cm.BuPu, edgecolor='k')
ax.set_xlabel(features[0])
ax.set_ylabel(features[1])
ax.set_zlabel('Partial dependence')
#  pretty init view
ax.view_init(elev=22, azim=122)
plt.colorbar(surf)
plt.suptitle('Partial dependence of house value on median\n'
             'age and average occupancy, with Gradient Boosting')
plt.subplots_adjust(top=0.9)

In [ ]:
plt.show()